# Transformation Equations:  ComCam <--> PanSTARRS-1 DR2

_Meagan N. Porter/Douglas L. Tucker_

_2025.6.27_

## Initial Setup

In [1]:
import numpy as np
import pandas as pd

import os
import sys
import glob
import math
import datetime

from collections import OrderedDict as odict

from astropy.io import fits
from astropy.table import Table

import fitsio

from scipy import interpolate
from scipy.optimize import leastsq

import healpy as hp

import plotly
from plotly.offline import download_plotlyjs, plot, iplot
import plotly.graph_objs as go

import matplotlib.pyplot as plt

%matplotlib inline

## Input Parameters

In [2]:
# List of bands on which to run the transform fit...
#bandList = ['g']
bandList = ['g', 'r', 'i', 'z', 'y']
    
# ComCam-->PS1 (toComCam=False) or PS1-->ComCam (toComCam=True)...
toComCam = True
#toComCam = False


# Color to fit against...
color_name_1 = 'gi'

# Name of color_name_1 as the label in the QA plots...
colorLabel_1 = '(g-i)'


if toComCam:
    # If PS1-->ComCam, then the color to fit against and its plot label are PS1...
    color_name_1 = """%s_ps1""" % (color_name_1)
    colorLabel_1 = """$%s_{ps1}$""" % colorLabel_1    
else:
    # If ComCam-->PS1, then the color to fit against and its plot label are ComCam...
    color_name_1 = """%s_ComCam""" % (color_name_1)
    colorLabel_1 = """$%s_{ComCam}$""" % colorLabel_1    


# Order of polynomial fits...
norder = 1

# Sigma-clipping parameters...
nsigma = 3.0
niter = 3

# Identify directory containing the match file...
matchDir = '/home/d/dltucker/DATA'

# Name of the match file in matchDir
matchFile = 'match_ComCam_PS1DR2_CDFS.fits'

# Base name of fit results output file...
if toComCam:
    resultsFileBaseName = 'transFit.ps1_to_ComCam'
else:
    resultsFileBaseName = 'transFit.ComCam_to_ps1'

# Base name of QA plot output files...
if toComCam:
    qaFileBaseName = 'qaPlot.ps1_to_ComCam.fit'
else:
    qaFileBaseName = 'qaPlot.ComCam_to_ps1.fit'

# Verbosity level (0, 1, 2, 3, ...)
verbose = 2

## Define Some Modules

In [3]:
# Kudos to Claude-3.5-Sonnet for improving on old outlier rejection code...

def poly_fit_with_sigma_clip(x, y, degree=1, sigma=3.0, maxiters=5):
    """
    Perform polynomial fit with iterative sigma clipping
    
    Parameters:
    -----------
    x : array-like
        Independent variable
    y : array-like 
        Dependent variable
    degree : int
        Degree of polynomial fit
    sigma : float
        Sigma clipping threshold
    maxiters : int
        Maximum number of sigma clipping iterations
        
    Returns:
    --------
    coeffs : array
        Polynomial coefficients
    mask : array
        Boolean mask indicating non-clipped points
    rms : float
        RMS of residuals
    """

    # Import relevant modules
    import numpy as np
    from astropy.stats import sigma_clip
    
    # Initial fit using all points
    x = np.asarray(x)
    y = np.asarray(y)
    mask = np.ones_like(x, dtype=bool)
    
    for _ in range(maxiters):
        # Fit polynomial to non-masked points
        coeffs = np.polyfit(x[mask], y[mask], degree)
        
        # Calculate residuals
        yfit = np.polyval(coeffs, x)
        residuals = y - yfit
        
        # Update mask with sigma clipping
        new_mask = ~sigma_clip(residuals, sigma=sigma).mask
        
        # Check for convergence
        if np.array_equal(mask, new_mask):
            break
            
        mask = new_mask
    
    # Calculate final RMS
    final_residuals = y[mask] - np.polyval(coeffs, x[mask])
    rms = np.sqrt(np.mean(final_residuals**2))
    
    return coeffs, mask, rms

In [4]:
##################################
#
# Define some functions for fitting dmag vs. color...
#
# These functions are based on a scripts found at 
# http://linuxgazette.net/115/andreasen.html (by Anders Andreasen)
# and at
# http://www.phy.uct.ac.za/courses/python/examples/fitresonance.py (University of Cape Town)


##################################

# Parametric function:  
#  p is the parameter vector; 
#  For fp1, we assume a polynomial function in one color...
def fp1(p,color1_array):
    #retValue = p[0] + p[1]*color1_array + p[2]*color1_array*color1_array
    norder = p.size-1
    retValue = p[0]
    for i in range(norder):
        retValue = retValue + p[i+1]*color1_array**(i+1)
    return retValue


##################################

# Error function:
def residuals1(p,color1_array,dmag_array):
    err = (dmag_array-fp1(p,color1_array))
    return err


##################################

# Fitting code:
def transformFit1(color1_array, dmag_array, norder=2, verbose=0):

    # Calculate the median of dmag for use as an initial guess
    # for the overall zeropoint offset..
    mdn = np.median( dmag_array, None )

    # Parameter names
    #pname = (['c_0', 'c_1', 'c_2'])
    pname = []
    for i in range(0,norder+1):
        pname.append("""c_%d""" % i)

    # Initial parameter values
    #p0 = [mdn, 0.0, 0.0]
    p0 = (1+norder)*[0.0]
    p0[0] = mdn

    if verbose > 0:
        print() 
        print( 'Initial parameter values:  ', p0)

    # Perform fit

    p,cov,infodict,mesg,ier = leastsq(residuals1, p0, 
                                      args=(color1_array, dmag_array), 
                                      maxfev=10000, full_output=1)

    if ( ier>=1 and ier <=4):
        if verbose > 0:  print("Converged")
    else:
        # Add an exception error or a non-zero return value?
        print("Not converged")
        print(mesg)


    # Calculate some descriptors of the fit 
    # (similar to the output from gnuplot 2d fits)

    chisq=sum(infodict['fvec']*infodict['fvec'])
    dof=len(dmag_array)-len(p)
    rms=math.sqrt(chisq/dof)
    
    if verbose > 0:
        print("Converged with chi squared ",chisq)
        print("degrees of freedom, dof ", dof)
        print("RMS of residuals (i.e. sqrt(chisq/dof)) ", rms)
        print("Reduced chisq (i.e. variance of residuals) ", chisq/dof)
        print()


    # uncertainties are calculated as per gnuplot, "fixing" the result
    # for non unit values of the reduced chisq.
    # values at min match gnuplot
    perr = []
    if verbose > 0:  
        print("Fitted parameters at minimum, with 68% C.I.:")
    for i,pmin in enumerate(p):
        if verbose > 0:  
            print("%-10s %13g +/- %13g   (%5f percent)" % (pname[i],pmin,math.sqrt(cov[i,i])*math.sqrt(chisq/dof),
                                                           100.*math.sqrt(cov[i,i])*math.sqrt(chisq/dof)/abs(pmin)))
        perr.append(math.sqrt(cov[i,i])*math.sqrt(chisq/dof))

    if verbose > 0: print()

    if verbose > 0:
        print( "Correlation matrix:")
        # correlation matrix close to gnuplot
        print( "               ",)
        for i in range(len(pname)): print( "%-10s" % (pname[i],),)
        print()
        for i in range(len(p)):
            print( "%-10s" % pname[i],)
            for j in range(i+1):
                print( "%10f" % (cov[i,j]/math.sqrt(cov[i,i]*cov[j,j]),),)
            #endfor
            print()
        #endfor
        print()
        print()
        print()
    
    return p, perr, rms


In [5]:
##################################

def createFitResultsOutputLine(norder, p, perr, rms, dmag_name, color1_name, color2_name=''):

    outputList = (2*(2*norder+1)+4)*[-9999.]
    outputList[0] = dmag_name
    outputList[1] = color1_name
    outputList[2] = color2_name
    for j in range(p.size):
        outputList[2*j+3] = p[j]
        outputList[2*j+4] = perr[j]
    outputList[2*(2*norder+1)+3] = rms
    outputLine = ','.join(map(str, outputList))
    return outputLine


##################################

def createFitResultsHeaderOutputLine(norder):

    outputList = (2*(2*norder+1)+4)*['c_']
    outputList[0] = 'dmag_name'
    outputList[1] = 'color1_name'
    outputList[2] = 'color2_name'
    for j in range(2*norder+1):
        outputList[2*j+3] = ("""c_%d""" % j)
        outputList[2*j+4] = ("""cerr_%d""" % j)
    outputList[2*(2*norder+1)+3] = 'rms'
    outputLine = ','.join(map(str, outputList))
    return outputLine


##################################

def transform1ColorQAPlots1(dmag, color1, res, norder, title, dmagName, colorLabel1, p, rms, outputFileName):

    # Prepare QA plots...
    #fig = plt.figure(figsize=(10,5))
    #fig = plt.figure(figsize=(40,20))
    fig = plt.figure(figsize=(20,10))
    fig.subplots_adjust(hspace=0.3)
    #fig.suptitle("This is a supertitle!")
    plt.rcParams.update({'font.size': 24})

   # We will exclude the lowest and highets 0.01% of color1, color2, 
    #  dmag, and residuals when plotting the QA figures...
    color1_desc = color1.describe(percentiles=[0.0001, 0.001, 0.01, 0.99, 0.999, 0.9999])
    dmag_desc = dmag.describe(percentiles=[0.0001, 0.001, 0.01, 0.99, 0.999, 0.9999])
    #res_desc = df.res.describe(percentiles=[0.0001, 0.001, 0.01, 0.99, 0.999, 0.9999])
    res_desc = res.describe(percentiles=[0.0001, 0.001, 0.01, 0.99, 0.999, 0.9999])
    #color1_min = color1_desc['1%']
    #color1_max = color1_desc['99%']
    #color1_min = color1_desc['min']
    #color1_max = color1_desc['max']
    #dmag_min = dmag_desc['1%']
    #dmag_max = dmag_desc['99%']
    #res_min = res_desc['1%']
    #res_max = res_desc['99%']
    color1_min = color1_desc['0.01%']
    color1_max = color1_desc['99.99%']
    dmag_min = dmag_desc['0.01%']
    dmag_max = dmag_desc['99.99%']
    res_min = res_desc['0.01%']
    res_max = res_desc['99.99%']

    # Plot 1:  Descriptive text...
    #plt.subplot(231)
    plt.subplot(221)
    if norder == 1:
        plot1Text = """%s \n\n%s = \n %.3f + \n %.3f*%s \n\n [rms: %.3f] \n\n [%.1f < %s < %.1f]""" % \
            (title, dmagName, p[0], p[1], colorLabel1, rms, color1_desc['min'], colorLabel1, color1_desc['max'])
    elif norder == 2:
        plot1Text = """%s \n\n%s = \n %.3f + \n %.3f*%s + \n %.3f*%s^2 \n\n [rms: %.3f] \n\n [%.1f < %s < %.1f]""" % \
            (title, dmagName, p[0], p[1], colorLabel1, p[2], colorLabel1, rms, color1_desc['min'], colorLabel1, color1_desc['max'])
    else:
        plot1Text = ''
    #plt.text(0.1,0.25,plot1Text)
    plt.text(0.1,0.00,plot1Text)
    plt.axis('off')

    
    # Plot 2:  2D hexbin histogram of dmag vs. color1...
    #plt.subplot(232) 
    plt.subplot(222)
    hb=plt.hexbin(color1, dmag, gridsize=100, bins='log', cmap='inferno')
    plt.axis([color1_min, color1_max, dmag_min, dmag_max])
    plt.xlabel(colorLabel1)
    plt.ylabel(dmagName)
    cb = fig.colorbar(hb)
    #cb.set_label('Number')
    cb.set_label('log10(N)')
    plt.grid(color='white')
    plt.grid(True)


    # Plot 3:  1d histogram of residuals...
    #plt.subplot(234) 
    plt.subplot(223) 
    #plt.hist(df.loc[:,'res'],bins=100)
    plt.hist(res,bins=100)
    plt.xlabel('residuals [mag]')
    plt.ylabel('Number')
    plt.grid(True)
    plt.grid(color='black')

    
    # Plot 4:  2d hexbin histogram of residuals vs. color1...
    #plt.subplot(235) 
    plt.subplot(224) 
    #hb = plt.hexbin(color1, df.loc[:,'res'], gridsize=100, cmap='inferno')
    hb = plt.hexbin(color1, res, gridsize=100, bins='log', cmap='inferno')
    plt.axis([color1_min, color1_max, res_min, res_max])
    plt.xlabel(colorLabel1)
    plt.ylabel('residuals [mag]')
    cb = plt.colorbar(hb)
    #cb.set_label('Number')
    cb.set_label('log10(N)')
    plt.grid(True)
    plt.grid(color='white')


    # Plot...
    plt.tight_layout()
    #plt.show()
    plt.savefig(outputFileName)

    return 0


##################################


## Read in Matched Catalog

In [6]:
matchFile = os.path.join(matchDir,matchFile)
print( matchFile)

/home/d/dltucker/DATA/match_ComCam_PS1DR2_CDFS.fits


In [7]:
# Check to make sure matchFile exists...
if os.path.isfile(matchFile)==False:
    print( """ERROR:  matchFile %s does not exist...""" % (matchFile))
if verbose > 0:
    print( 'matchFile: ', matchFile)


matchFile:  /home/d/dltucker/DATA/match_ComCam_PS1DR2_CDFS.fits


In [8]:
tab = Table.read(matchFile, format='fits')
tab

tractId,field,coord_ra,coord_dec,detect_isPrimary,u_psfFlux,u_psfFluxErr,u_ap12Flux,u_ap12FluxErr,u_extendedness,u_psfFlux_flag,g_psfFlux,g_psfFluxErr,g_ap12Flux,g_ap12FluxErr,g_extendedness,g_psfFlux_flag,r_psfFlux,r_psfFluxErr,r_ap12Flux,r_ap12FluxErr,r_extendedness,r_psfFlux_flag,i_psfFlux,i_psfFluxErr,i_ap12Flux,i_ap12FluxErr,i_extendedness,i_psfFlux_flag,z_psfFlux,z_psfFluxErr,z_ap12Flux,z_ap12FluxErr,z_extendedness,z_psfFlux_flag,y_psfFlux,y_psfFluxErr,y_ap12Flux,y_ap12FluxErr,y_extendedness,y_psfFlux_flag,objName,raMean,decMean,raMeanErr,decMeanErr,qualityFlag,gMeanPSFMag,gMeanPSFMagErr,gMeanPSFMagNpt,rMeanPSFMag,rMeanPSFMagErr,rMeanPSFMagNpt,iMeanPSFMag,iMeanPSFMagErr,iMeanPSFMagNpt,zMeanPSFMag,zMeanPSFMagErr,zMeanPSFMagNpt,yMeanPSFMag,yMeanPSFMagErr,yMeanPSFMagNpt,rMeanKronMag,rMeanKronMagErr,nDetections,ng,nr,ni,nz,ny,gFlags,gQfPerfect,rFlags,rQfPerfect,iFlags,iQfPerfect,zFlags,zQfPerfect,yFlags,yQfPerfect,primaryDetection,bestDetection,Separation
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,deg,mas,mas,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,,,,,,,,,,,,,,,,,,arcsec
int16,bytes15,float64,float64,bool,float64,float64,float64,float64,float32,bool,float64,float64,float64,float64,float32,bool,float64,float64,float64,float64,float32,bool,float64,float64,float64,float64,float32,bool,float64,float64,float64,float64,float32,bool,float64,float64,float64,float64,float32,bool,bytes21,float64,float64,float32,float32,uint8,float32,float32,int16,float32,float32,int16,float32,float32,int16,float32,float32,int16,float32,float32,int16,float32,float32,int16,int16,int16,int16,int16,int16,int32,float32,int32,float32,int32,float32,int32,float32,int32,float32,uint8,uint8,float64
4849,ECDFS,53.27198574221387,-28.674615827805333,True,13286.0366708119,412.0371394823248,14683.306640625,771.7345581054688,0.0,False,28228.682373877586,94.24684984697546,28579.15625,147.3459930419922,0.0,False,40284.77862636537,176.57453634470306,40462.47265625,341.0738525390625,0.0,False,45304.27731117351,202.92832507957067,45365.5,399.6529235839844,0.0,False,48170.47049675424,352.13372978390413,48299.59375,787.4659423828125,0.0,False,--,--,--,--,--,True,PSO J053.2720-28.6746,53.27196779195253,-28.674607382095555,0.0038724404,0.003842568,52,20.2413,0.02725,12,19.8377,0.013462,10,19.7084,0.017276,23,19.7509,0.021025,10,19.6409,0.046909,7,19.9437,0.037949,64,12,10,24,11,7,115000,0.999737,115000,0.998922,115000,0.999662,115000,0.999726,115000,0.999342,1,1,0.06433386108875619
4849,ECDFS,53.32503490653086,-28.69850559504483,True,--,--,105.12812042236328,inf,--,True,88409.62232343505,202.8594636785005,88028.3203125,248.21238708496094,0.0,False,138202.43287938446,283.015352262914,134803.25,413.2403869628906,0.0,False,158709.10786182803,335.7959591269237,157858.453125,505.9420471191406,0.0,False,--,--,3117.481689453125,1110.4119873046875,--,True,--,--,--,--,--,True,PSO J053.3250-28.6985,53.32502525198759,-28.69849493660518,0.0017170375,0.0017088106,52,18.9806,0.013506,8,18.5692,0.005776,8,18.3848,0.008247,25,18.3334,0.008049,7,18.2719,0.013961,9,18.6736,0.013443,64,9,9,26,8,12,115000,0.999151,115000,0.999358,115000,0.999603,115000,0.999487,115000,0.999322,1,1,0.049007486633848726
4849,ECDFS,53.27836390943371,-28.686723873589237,True,2788.835404228934,373.39300678308064,3341.12744140625,763.09814453125,1.0,False,25829.309855406915,92.9626519491877,26075.66796875,147.3630828857422,0.0,False,80205.85267200095,205.32933100507614,80735.2578125,352.82635498046875,0.0,False,232223.648994296,378.8379460235678,232710.78125,525.6046752929688,0.0,False,367084.84696123755,817.0113297924988,365145.875,1193.2911376953125,0.0,False,--,--,--,--,--,True,PSO J053.2784-28.6867,53.27834934186566,-28.686692096520066,0.0013673233,0.0013519784,60,20.3157,0.034451,10,19.1718,0.011885,10,17.9977,0.00465,18,17.5143,0.006109,9,17.2658,0.005469,9,19.242,0.010498,63,10,10,24,9,10,115000,0.999504,115000,0.998803,115000,0.99964,115000,0.999689,115000,0.99954,1,1,0.12330186439086072
484

In [9]:
df = tab.to_pandas()
df

,tractId,field,coord_ra,coord_dec,detect_isPrimary,u_psfFlux,u_psfFluxErr,u_ap12Flux,u_ap12FluxErr,u_extendedness,...,rQfPerfect,iFlags,iQfPerfect,zFlags,zQfPerfect,yFlags,yQfPerfect,primaryDetection,bestDetection,Separation
0,4849,b'ECDFS',53.271986,-28.674616,True,13286.036671,412.037139,14683.306641,771.734558,0.0,...,0.998922,115000,0.999662,115000,0.999726,115000,0.999342,1,1,0.064334
1,4849,b'ECDFS',53.325035,-28.698506,True,NaN,NaN,105.128120,inf,NaN,...,0.999358,115000,0.999603,115000,0.999487,115000,0.999322,1,1,0.049007
2,4849,b'ECDFS',53.278364,-28.686724,True,2788.835404,373.393007,3341.127441,763.098145,1.0,...,0.998803,115000,0.999640,115000,0.999689,115000,0.999540,1,1,0.123302
3,4849,b'ECDFS',53.285193,-28.677181,True,12811.390078,412.231971,13945.104492,772.934509,1.0,...,0.999416,115000,0.999684,115000,0.998863,115000,0.999643,1,1,0.144540
4,4849,b'ECDFS',53.218667,-28.689860,True,8488.339090,452.516364,7462.151855,852.762085,1.0,...,0.998766,115000,0.999591,115000,0.999100,115000,0.999576,1,1,0.070754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608,4848,b'ECDFS',52.512609,-28.272698,True,976.611109,148.456380,3376.368896,528.876587,NaN,...,0.999850,115000,0.999495,115000,0.999339,115000,0.998865,1,1,0.221185
2609,4848,b'ECDFS',52.463471,-28.268926,True,NaN,NaN,1559.280518,21.253164,NaN,...,0.999835,115000,0.999689,115000,0.999387,115000,0.999580,1,1,0.350573
2610,4848,b'ECDFS',52.463330,-28.271611,True,NaN,NaN,1559.291992,21.285826,NaN,...,0.999462,115000,0.999418,115000,0.999666,115000,0.999356,1,1,0.008624
2611,4848,b'ECDFS',52.511222,-28.270058,True,493.956061,180.460181,2302.926270,549.716431,NaN,...,0.999275,115000,0.999348,115000,0.999060,115000,0.998892,1,1,0.058266


In [10]:
#df['u_psfMag'] = -2.5*np.log10(df['u_psfFlux']) + 31.4
df['g_psfMag'] = -2.5*np.log10(df['g_psfFlux']) + 31.4
df['r_psfMag'] = -2.5*np.log10(df['r_psfFlux']) + 31.4
df['i_psfMag'] = -2.5*np.log10(df['i_psfFlux']) + 31.4
df['z_psfMag'] = -2.5*np.log10(df['z_psfFlux']) + 31.4
df['y_psfMag'] = -2.5*np.log10(df['y_psfFlux']) + 31.4

In [11]:
#df['u_psfMagErr'] = 1.086*df['u_psfFluxErr']/df['u_psfFlux']
df['g_psfMagErr'] = 1.086*df['g_psfFluxErr']/df['g_psfFlux']
df['r_psfMagErr'] = 1.086*df['r_psfFluxErr']/df['r_psfFlux']
df['i_psfMagErr'] = 1.086*df['i_psfFluxErr']/df['i_psfFlux']
df['z_psfMagErr'] = 1.086*df['z_psfFluxErr']/df['z_psfFlux']
df['y_psfMagErr'] = 1.086*df['y_psfFluxErr']/df['y_psfFlux']

In [12]:
for col in df.columns:
    print(col)

tractId
field
coord_ra
coord_dec
detect_isPrimary
u_psfFlux
u_psfFluxErr
u_ap12Flux
u_ap12FluxErr
u_extendedness
u_psfFlux_flag
g_psfFlux
g_psfFluxErr
g_ap12Flux
g_ap12FluxErr
g_extendedness
g_psfFlux_flag
r_psfFlux
r_psfFluxErr
r_ap12Flux
r_ap12FluxErr
r_extendedness
r_psfFlux_flag
i_psfFlux
i_psfFluxErr
i_ap12Flux
i_ap12FluxErr
i_extendedness
i_psfFlux_flag
z_psfFlux
z_psfFluxErr
z_ap12Flux
z_ap12FluxErr
z_extendedness
z_psfFlux_flag
y_psfFlux
y_psfFluxErr
y_ap12Flux
y_ap12FluxErr
y_extendedness
y_psfFlux_flag
objName
raMean
decMean
raMeanErr
decMeanErr
qualityFlag
gMeanPSFMag
gMeanPSFMagErr
gMeanPSFMagNpt
rMeanPSFMag
rMeanPSFMagErr
rMeanPSFMagNpt
iMeanPSFMag
iMeanPSFMagErr
iMeanPSFMagNpt
zMeanPSFMag
zMeanPSFMagErr
zMeanPSFMagNpt
yMeanPSFMag
yMeanPSFMagErr
yMeanPSFMagNpt
rMeanKronMag
rMeanKronMagErr
nDetections
ng
nr
ni
nz
ny
gFlags
gQfPerfect
rFlags
rQfPerfect
iFlags
iQfPerfect
zFlags
zQfPerfect
yFlags
yQfPerfect
primaryDetection
bestDetection
Separation
g_psfMag
r_psfMag
i_psfMag
z

In [13]:
# Rename columns...
df.rename(columns={'coord_ra':'RA_ComCam',
                   'coord_dec':'DEC_ComCam',
                   'g_psfMag':'g_ComCam',
                   'r_psfMag':'r_ComCam',
                   'i_psfMag':'i_ComCam',
                   'z_psfMag':'z_ComCam',
                   'y_psfMag':'y_ComCam',
                   'g_psfMagErr':'g_err_ComCam',
                   'r_psfMagErr':'r_err_ComCam',
                   'i_psfMagErr':'i_err_ComCam',
                   'z_psfMagErr':'z_err_ComCam',
                   'y_psfMagErr':'y_err_ComCam',
                   'gMeanPSFMag':'g_ps1',
                   'rMeanPSFMag':'r_ps1',
                   'iMeanPSFMag':'i_ps1',
                   'zMeanPSFMag':'z_ps1',
                   'yMeanPSFMag':'y_ps1',
                   'gMeanPSFMagErr':'g_err_ps1',
                   'rMeanPSFMagErr':'r_err_ps1',
                   'iMeanPSFMagErr':'i_err_ps1',
                   'zMeanPSFMagErr':'z_err_ps1',
                   'yMeanPSFMagErr':'y_err_ps1'
                  },inplace=True)

df.head(5)

,tractId,field,RA_ComCam,DEC_ComCam,detect_isPrimary,u_psfFlux,u_psfFluxErr,u_ap12Flux,u_ap12FluxErr,u_extendedness,...,g_ComCam,r_ComCam,i_ComCam,z_ComCam,y_ComCam,g_err_ComCam,r_err_ComCam,i_err_ComCam,z_err_ComCam,y_err_ComCam
0,4849,b'ECDFS',53.271986,-28.674616,True,13286.036671,412.037139,14683.306641,771.734558,0.0,...,20.273273,19.887148,19.759652,19.693048,NaN,0.003626,0.004760,0.004864,0.007939,NaN
1,4849,b'ECDFS',53.325035,-28.698506,True,NaN,NaN,105.128120,inf,NaN,...,19.033751,18.548711,18.398495,NaN,NaN,0.002492,0.002224,0.002298,NaN,NaN
2,4849,b'ECDFS',53.278364,-28.686724,True,2788.835404,373.393007,3341.127441,763.098145,1.0,...,20.369718,19.139485,17.985234,17.488084,NaN,0.003909,0.002780,0.001772,0.002417,NaN
3,4849,b'ECDFS',53.285193,-28.677181,True,12811.390078,412.231971,13945.104492,772.934509,1.0,...,19.836505,19.270730,19.046171,18.929933,NaN,0.002752,0.003018,0.003495,0.006447,NaN
4,4849,b'ECDFS',53.218667,-28.689860,True,8488.339090,452.516364,7462.151855,852.762085,1.0,...,20.484508,20.022335,19.873060,19.823289,NaN,0.004921,0.006897,0.004440,0.006228,NaN


## Add Columns to Matched Catalog Data Frame

In [14]:
# Add color columns...
df.loc[:,'gr_ComCam'] = df.loc[:,'g_ComCam'] - df.loc[:,'r_ComCam']
df.loc[:,'ri_ComCam'] = df.loc[:,'r_ComCam'] - df.loc[:,'i_ComCam']
df.loc[:,'iz_ComCam'] = df.loc[:,'i_ComCam'] - df.loc[:,'z_ComCam']
df.loc[:,'zy_ComCam'] = df.loc[:,'z_ComCam'] - df.loc[:,'y_ComCam']
df.loc[:,'gi_ComCam'] = df.loc[:,'g_ComCam'] - df.loc[:,'i_ComCam']

df.loc[:,'gr_ps1'] = df.loc[:,'g_ps1'] - df.loc[:,'r_ps1']
df.loc[:,'ri_ps1'] = df.loc[:,'r_ps1'] - df.loc[:,'i_ps1']
df.loc[:,'iz_ps1'] = df.loc[:,'i_ps1'] - df.loc[:,'z_ps1']
df.loc[:,'zy_ps1'] = df.loc[:,'z_ps1'] - df.loc[:,'y_ps1']
df.loc[:,'gi_ps1'] = df.loc[:,'g_ps1'] - df.loc[:,'i_ps1']


In [15]:
# Insert dmag column...
df.loc[:,'dmag'] = -9999.

## Create Initial Mask

In [16]:
mask = ( ( df[color_name_1] > -1. ) & ( df[color_name_1] < 4.0 ) )

## Make Backup Copies of Initial Mask and Original Data Frame

In [17]:
# Make a backup copy of original df...
df_orig = df.copy()

# Make a backup copy of original mask...
mask_orig = mask.copy()

In [18]:
#XXXX

In [19]:
# Create results output file...
resultsFile = """%s.dmag.%s.norder%d.csv""" % (resultsFileBaseName, color_name_1, norder)

# Open fit results output file...
try:
    fout = open(resultsFile, 'w')
except IOError:
    sys.exit('Unable to write to file ' + resultsFile)

# Write header to fit results output file...
hdr = createFitResultsHeaderOutputLine(norder)
fout.write(hdr+'\n')

for band in bandList:
    
    print() 
    print() 
    print() 
    print( "# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # ")
    print( band)
    print( "# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # ")
    print() 
    
    magName_ComCam = """%s_ComCam""" % (band)
    magErrName_ComCam = """%s_err_ComCam""" % (band)
    magName_ps1 = """%s_ps1""" % (band)
    magErrName_ps1 = """%s_err_ps1""" % (band)
    
    # Grab the original version of df from the backup copy...
    df = df_orig.copy()

    # Grab the original version of mask from the backup copy...
    mask = mask_orig.copy()

    # Update dmag column for {$band}_des - {$band}_ComCam...
    if toComCam:
        df.loc[:,'dmag'] = df.loc[:,magName_ComCam] - df.loc[:,magName_ps1]
    else:
        df.loc[:,'dmag'] = df.loc[:,magName_ps1] - df.loc[:,magName_ComCam]
        
    # Update mask...
    mask1 = abs(df['dmag']) <= 10.
    mask2 = abs(df[magErrName_ComCam]) <= 0.02
    mask3 = abs(df[magErrName_ps1]) <= 0.01
    mask = mask & mask1 & mask2 & mask3

    # Iterate, with sigma-clipping...
    for i in range(niter):

        iiter = i + 1
        if verbose > 0:
            print( """   iter%d...""" % ( iiter ))

        # make a copy of original df, overwriting the old one...
        df = df[mask].copy()

        # Identify dmag and color1 series...
        dmag =  df.loc[:,'dmag']
        color1 = df.loc[:,color_name_1]

        print(color1)
        print(dmag)
        
        # Perform fit...
        p,perr,rms = transformFit1(color1, dmag, norder, verbose)
        df.loc[:,'res'] = residuals1(p, color1, dmag)

        # Identify outliers...|
        stddev = df['res'].std()
        mask = (np.abs(df.res)< nsigma*stddev)


    # Output results to the results file...
    outputLine = createFitResultsOutputLine(2, p, perr, rms, band, color_name_1)
    fout.write(outputLine+'\n')
    
    # Create title/names for use in QA plots...
    if toComCam:
        title = """$%s_{ps1}$ --> $%s_{ComCam}$""" % (band, band)
        dmagName = """$%s_{ComCam} - %s_{ps1}$""" % (band, band)
    else:
        title = """$%s_{ComCam}$ --> $%s_{ps1}$""" % (band, band)
        dmagName = """$%s_{ps1} - %s_{ComCam}$""" % (band, band)
    
    # Create QA plots...
    res =  df.loc[:,'res']
    outputFileName = """%s.dmag_%s.%s.norder%d.qa1.png""" % (qaFileBaseName, band, color_name_1, norder)
    status = transform1ColorQAPlots1(dmag, color1, res, norder, title, dmagName, colorLabel_1, p, rms, outputFileName)

fout.close()




# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
g
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

   iter1...
5       0.332201
6       1.789299
7       0.692200
12      1.450701
16      0.554001
          ...   
2589    0.948300
2596    2.001101
2605    0.645299
2606    0.861000
2609    0.446199
Name: gi_ps1, Length: 700, dtype: float32
5       0.022756
6       0.090311
7       0.044147
12      0.054465
16      0.041264
          ...   
2589    0.021683
2596    0.084186
2605    0.051134
2606    0.049288
2609    0.013909
Name: dmag, Length: 700, dtype: float64

Initial parameter values:   [np.float64(0.05378856442423796), 0.0]
Converged
Converged with chi squared  3.4603057520988756
degrees of freedom, dof  698
RMS of residuals (i.e. sqrt(chisq/dof))  0.0704092188393172
Reduced chisq (i.e. variance of residuals)  0.004957458097562859

Fitted parameters at minimum, with 68% C.I.:


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Example with plotting
import matplotlib.pyplot as plt

band = 'g'

magName_ComCam = """%s_ComCam""" % (band)
magErrName_ComCam = """%s_err_ComCam""" % (band)
magName_ps1 = """%s_ps1""" % (band)
magErrName_ps1 = """%s_err_ps1""" % (band)
    
# Grab the original version of df from the backup copy...
df = df_orig.copy()

# Grab the original version of mask from the backup copy...
mask = mask_orig.copy()

# Update dmag column for {$band}_des - {$band}_ComCam...
if toComCam:
    df.loc[:,'dmag'] = df.loc[:,magName_ComCam] - df.loc[:,magName_ps1]
else:
    df.loc[:,'dmag'] = df.loc[:,magName_ps1] - df.loc[:,magName_ComCam]
        
# Update mask...
mask1 = abs(df['dmag']) <= 10.
mask2 = abs(df[magErrName_ComCam]) <= 0.02
mask3 = abs(df[magErrName_ps1]) <= 0.01
mask = mask & mask1 & mask2 & mask3

# make a copy of original df, overwriting the old one...
df = df[mask].copy()

# Identify dmag and color1 series...
dmag =  df.loc[:,'dmag']
color1 = df.loc[:,color_name_1]


color1_array = color1
dmag_array = dmag

# Perform the fit
coeffs, mask, rms = poly_fit_with_sigma_clip(color1_array, dmag_array, degree=1)

# Generate smooth curve for plotting
x_smooth = np.linspace(min(color1_array), max(color1_array), 100)
y_smooth = np.polyval(coeffs, x_smooth)

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(color1_array[mask], dmag_array[mask], label='Used points')
plt.scatter(color1_array[~mask], dmag_array[~mask], color='red', label='Rejected points')
plt.plot(x_smooth, y_smooth, 'k-', label='Fit')
plt.xlabel('Color')
plt.ylabel('dmag')
plt.legend()
plt.title(f'Polynomial fit (RMS = {rms:.3f})')
plt.show()

# Print coefficients
for i, c in enumerate(coeffs):
    print(f'c_{len(coeffs)-i-1} = {c:.6f}')